In [ ]:
### TODO Last Updated - 11.09.25 1231Z 
### Continue to work on CRS string issue (see ChatGPT conversation on this topic)

### TODO, short term (by 11.16)
### Continue to work on Conda import of rioxarray & reprojection of RTMA to HRRR grid instea of interp (see ChatGPT conversation on this topic)
### TODO, medium term (by 11.25)
### Develop test using Codex that makes sure that calculations are correct

### TODO, long term (12.09)
### Develop method for testing NAM3K against RTMA
### Develop method for testing HRRR against NAM3K
### Develop method for testing HRRR/NAM3K against URMA

from herbie import Herbie
from herbie.toolbox import EasyMap, pc
from herbie import paint
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta
import rioxarray

In [9]:
date = input("Enter date (YYYY-MM-DD): ")
init_hour = int(input("Enter initialization hour (HH): "))
forecast = int(input("Enter forecast hour (0..N): "))

# Build the HRRR cycle datetime (initialization time)
cycle_dt = datetime.fromisoformat(f"{date} {init_hour:02d}:00")

# Compute the forecast valid time (handles day/month/year rollovers)
valid_dt = cycle_dt + timedelta(hours=forecast)

# Optional: a display string if you need it
displayhour = valid_dt.strftime("%H:%M")

# Compute display hour safely
displayhour = (forecast + init_hour) % 24

# HRRR: pass the cycle time + forecast hour
hrrr = Herbie(
    cycle_dt,
    model="hrrr",
    product="sfc",
    fxx=forecast
)

# RTMA: pass the *valid* analysis time so it matches HRRR's valid time
rtma = Herbie(
    valid_dt,
    model="rtma",
    product="anl"
)


✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Nov-12 12:00 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=rtma ┊ product=anl ┊ 2025-Nov-12 18:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


In [10]:
ds_hrrr = hrrr.xarray("TMP:2 m above")
ds_hrrr



<xarray.Dataset> Size: 38MB
Dimensions:              (y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 8B 2025-11-12T12:00:00
    step                 timedelta64[ns] 8B 06:00:00
    heightAboveGround    float64 8B 2.0
    latitude             (y, x) float64 15MB 21.14 21.15 21.15 ... 47.85 47.84
    longitude            (y, x) float64 15MB 237.3 237.3 237.3 ... 299.0 299.1
    valid_time           datetime64[ns] 8B 2025-11-12T18:00:00
    gribfile_projection  object 8B None
Dimensions without coordinates: y, x
Data variables:
    t2m                  (y, x) float32 8MB 296.1 296.1 296.1 ... 276.1 276.1
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 sfc
    description:             High-Resolution Rapid Refresh - CONUS
    remote_grib:             https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr....
    local_grib:              /Users/john_b_needham/data/hrrr/20251112/subset_...
    search:                  TMP:2 m above

In [11]:
ds_rtma = rtma.xarray("TMP:2 m above")
ds_rtma

<xarray.Dataset> Size: 75MB
Dimensions:              (y: 1597, x: 2345)
Coordinates:
    time                 datetime64[ns] 8B 2025-11-12T18:00:00
    step                 timedelta64[ns] 8B 00:00:00
    heightAboveGround    float64 8B 2.0
    latitude             (y, x) float64 30MB 19.23 19.23 19.24 ... 54.38 54.37
    longitude            (y, x) float64 30MB 233.7 233.7 233.8 ... 300.9 301.0
    valid_time           datetime64[ns] 8B 2025-11-12T18:00:00
    gribfile_projection  object 8B None
Dimensions without coordinates: y, x
Data variables:
    t2m                  (y, x) float32 15MB 296.7 296.7 296.7 ... 273.6 273.3
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          4
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   rtma
    product:                 anl
    description:             CONUS Real-Time Mesoscale Analysis (RTMA)
    remote_grib:             https://noaa-rtma-pds.s3.amazonaws.com/rtma2p5.2...
    local_grib:              /Users/john_b_needham/data/rtma/20251112/subset_...
    search:                  TMP:2 m above

In [14]:
# 1) Ensure both have CRS set (PROJ strings). Example only — replace with your actual CRS!
# You can often read these from file attrs (cfgrib sets Lambert Conformal params).
rtma = ds_rtma.copy()
hrrr = ds_hrrr.copy()

rtma['t2m'] = rtma['t2m'].rio.write_crs("EPSG:4326")  # if rtma is in lat/lon
hrrr['t2m'] = hrrr['t2m'].rio.write_crs(hrrr.attrs)["gribfile_projection"] # HRRR's Lambert Conformal CRS string

# 2) Reproject RTMA to match HRRR grid (same CRS, transform, resolution, shape)
rtma_on_hrrr = rtma['t2m'].rio.reproject_match(hrrr['t2m'])

# 3) Difference in °F
tempdiff_f = (hrrr['t2m'] - rtma_on_hrrr) * 9/5


CRSError: Invalid projection: {"GRIB_edition": 2, "GRIB_centre": "kwbc", "GRIB_centreDescription": "US National Weather Service - NCEP", "GRIB_subCentre": 0, "Conventions": "CF-1.7", "institution": "US National Weather Service - NCEP", "model": "hrrr", "product": "sfc", "description": "High-Resolution Rapid Refresh - CONUS", "remote_grib": "https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20251112/conus/hrrr.t12z.wrfsfcf06.grib2", "local_grib": "/Users/john_b_needham/data/hrrr/20251112/subset_28b2fd54__hrrr.t12z.wrfsfcf06.grib2", "search": "TMP:2 m above"}: (Internal Proj Error: proj_create: Missing "type" key)

In [ ]:
rtma_on_hrrr.sizes["x"] == ds_hrrr.sizes["x"]  # -> 1799

In [ ]:
rtma_on_hrrr.sizes["y"] == ds_hrrr.sizes["y"]  # -> 1059

In [ ]:
rtma_on_hrrr = rtma_on_hrrr.assign_coords(
    latitude=ds_hrrr.latitude,
    longitude=ds_hrrr.longitude,
)

In [ ]:
rtma_on_hrrr["latitude"] == ds_hrrr["latitude"]  # -> True

In [ ]:
rtma_on_hrrr["longitude"] == ds_hrrr["longitude"]  #-> True

In [ ]:
print(ds_hrrr.x.attrs.get("units"), ds_hrrr.y.attrs.get("units"))
print(ds_rtma.herbie.crs)

In [ ]:
# 2) Use HRRR lon/lat + wrap
lon = ds_hrrr["longitude"]
lat = ds_hrrr["latitude"]
lon = xr.where(lon > 180, lon - 360, lon)

# 3) Make a Cartopy map (no EasyMap)
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-95))  # pick your map proj
ax.add_feature(cfeature.BORDERS, linewidth=0.6)
ax.add_feature(cfeature.STATES, linewidth=0.4)
ax.coastlines(resolution="50m", linewidth=0.6)

hrrr_f = (ds_hrrr.t2m - 273.15) * 9/5 + 32

# 4) Plot (coordinates are lon/lat centers)
pc = ccrs.PlateCarree()
p = ax.pcolormesh(
    lon, lat, hrrr_f,
    cmap="coolwarm",
    transform=pc,         # data coords
    shading="nearest",     # treat lon/lat as cell centers
)

cb = plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.02, shrink=0.85)
cb.set_label("HRRR 2m Temperature")
ax.set_title("HRRR 2m Temperature of " + str(date) + " " + str(init_hour) + "Z run at forecast hour " + str(forecast))
plt.show()

In [ ]:
# 2) Use HRRR lon/lat + wrap
lon = ds_rtma["longitude"]
lat = ds_rtma["latitude"]
lon = xr.where(lon > 180, lon - 360, lon)

# 3) Make a Cartopy map (no EasyMap)
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-95))  # pick your map proj
ax.add_feature(cfeature.BORDERS, linewidth=0.6)
ax.add_feature(cfeature.STATES, linewidth=0.4)
ax.coastlines(resolution="50m", linewidth=0.6)

rtma_f = (ds_rtma.t2m - 273.15) * 9/5 + 32

# 4) Plot (coordinates are lon/lat centers)
pc = ccrs.PlateCarree()
p = ax.pcolormesh(
    lon, lat, rtma_f,
    cmap="coolwarm",
    transform=pc,         # data coords
    shading="nearest",    # treat lon/lat as cell centers
)

cb = plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.02, shrink=0.85)
cb.set_label("RTMA 2m Temperature")
ax.set_title("RTMA 2m Temperature at " + str(date) + " " + str(displayhour) + "Z")
plt.show()

In [ ]:
diff_f = (ds_hrrr.t2m - rtma_on_hrrr.t2m) * 9/5
print("-F:", float(diff_f.min()), float(diff_f.max()), float(diff_f.mean()))

In [ ]:
print(ds_hrrr.t2m.dtype, ds_hrrr.t2m.encoding)
print(rtma_on_hrrr.t2m.dtype, rtma_on_hrrr.t2m.encoding)
print(ds_hrrr.t2m.attrs.get("units"), rtma_on_hrrr.t2m.attrs.get("units"))

# Sanity ranges (real temps should be ~200–330 K)
print((ds_hrrr.t2m.quantile([0, .5, 1]).values))
print((ds_rtma.t2m.quantile([0, .5, 1]).values))
print((rtma_on_hrrr.t2m.quantile([0, .5, 1]).values))

In [ ]:
print("coords equal?", np.allclose(ds_hrrr.x, rtma_on_hrrr.x) and np.allclose(ds_hrrr.y, rtma_on_hrrr.y))

# but *also* check if lats/lons differ at the same indices:
if {'lat','lon'}.issubset(ds_hrrr):
    same_latlon = (np.allclose(ds_hrrr.lat, rtma_on_hrrr.lat, equal_nan=True) and
                   np.allclose(ds_hrrr.lon, rtma_on_hrrr.lon, equal_nan=True))
    print("lat/lon fields equal?", same_latlon)
else:
    print("lat/lon fields not equal!")

In [ ]:
# Compute HRRR - RTMA difference in Fahrenheit
tempdiff_f = (ds_hrrr.t2m - rtma_on_hrrr.t2m) * 9/5

print(tempdiff_f)

# 2) Use HRRR lon/lat + wrap
lon = ds_hrrr["longitude"]
lat = ds_hrrr["latitude"]
lon = xr.where(lon > 180, lon - 360, lon)

# 3) Make a Cartopy map
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-95))  # pick your map proj
ax.add_feature(cfeature.BORDERS, linewidth=0.6)
ax.add_feature(cfeature.STATES, linewidth=0.4)
ax.coastlines(resolution="50m", linewidth=0.6)

# 4) Plot (coordinates are lon/lat centers)
pc = ccrs.PlateCarree()
p = ax.pcolormesh(
    lon, lat, tempdiff_f,
    cmap="coolwarm",
    transform=pc,         # data coords
    shading="nearest",     # treat lon/lat as cell centers
    vmin=-20, vmax=20
)

cb = plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.02, shrink=0.85)
cb.set_label("HRRR − RTMA 2m Temperature (F)")
ax.set_title("HRRR − RTMA: 2m Temperature Difference at " + str(date) + " " + str(displayhour) + "Z in F")
plt.show()